In [1]:
import pyequion
import sympy
%load_ext Cython

## Create a system and save it to a C file

In [2]:
sys_eq = pyequion.create_equilibrium(['NaHCO3', 'CaCl2'])
pyequion.save_res_to_c_code(sys_eq, 'dummy', 'calc_cnv_res_equilibrium_NaHCO3_CaCl2', 
    # fixed_temperature=25.0
)


Check the file system for the created header and source files

## Generating Cython module for calling the C code

Reference: https://www.sympy.org/scipy-2017-codegen-tutorial/

1. Firstly a magic cell is used to create the build configuration file
1. Nextly, the cython file that will bridge the python interpreter with the c-function is defined and make it available to python interpreter

In [3]:
%%writefile calc_cnv_res_equilibrium_NaHCO3_CaCl2.pyxbld
import numpy

#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'calc_cnv_res_equilibrium_NaHCO3_CaCl2.c'],
                     include_dirs=['.', numpy.get_include()])

Writing calc_cnv_res_equilibrium_NaHCO3_CaCl2.pyxbld


In [4]:
%%cython_pyximport calc_cnv_res_equilibrium_NaHCO3_CaCl2
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API

# here we just replicate the function signature from the header
cdef extern from "calc_cnv_res_equilibrium_NaHCO3_CaCl2.h":
    void calc_cnv_res_equilibrium_NaHCO3_CaCl2(double T, double *concs, double *x, double *res)

# here is the "wrapper" signature that conforms to the odeint interface
def cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(double T, cnp.ndarray[cnp.double_t, ndim=1] concs, cnp.ndarray[cnp.double_t, ndim=1] x):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] res = np.empty(x.size, dtype=np.double)
    # now call the C function
    calc_cnv_res_equilibrium_NaHCO3_CaCl2(<double> T, <double *> concs.data, <double *> x.data, <double *> res.data)
    # return the result
    return res

In [5]:
sol = pyequion.solve_solution({'NaHCO3': 10, 'CaCl2': 5})
cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(25.0, np.array([10.0, 5.0]), sol.x)

array([-2.84955833e+02,  7.53626726e+00, -4.94878719e-02,  7.38949034e+02,
       -7.22534159e+01, -1.39843274e+00, -1.99624148e+03, -4.83136100e-01,
        8.58372158e+02, -8.45007295e+02,  9.99000000e+00,  9.99000000e+00,
        4.99500000e+00,  9.99000000e+00, -1.73472348e-18])

## Generating C Code for the Jacobian

In [3]:
pyequion.save_jacobian_of_res_to_c_code(sys_eq, 'dummy', 'calc_cnv_res_equilibrium_NaHCO3_CaCl2', 
    # fixed_temperature=25.0
)


                                                                                                                                                                                                                                                                                      -4.60517018598809*10.0**x[6, 0],                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [4]:
%%writefile calc_cnv_jac_equilibrium_NaHCO3_CaCl2.pyxbld
import numpy

#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'calc_cnv_jac_equilibrium_NaHCO3_CaCl2.c'],
                     include_dirs=['.', numpy.get_include()])

Overwriting calc_cnv_jac_equilibrium_NaHCO3_CaCl2.pyxbld


In [8]:
%%cython_pyximport calc_cnv_jac_equilibrium_NaHCO3_CaCl2
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API

# here we just replicate the function signature from the header
cdef extern from "calc_cnv_jac_equilibrium_NaHCO3_CaCl2.h":
    void calc_cnv_jac_equilibrium_NaHCO3_CaCl2(double T, double *x, double *res)

# here is the "wrapper" signature that conforms to the odeint interface
def cy_calc_cnv_jac_equilibrium_NaHCO3_CaCl2(double T, cnp.ndarray[cnp.double_t, ndim=1] x):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] J = np.empty((x.size*x.size), dtype=np.double)
    # now call the C function
    calc_cnv_jac_equilibrium_NaHCO3_CaCl2(<double> T, <double *> x.data, <double *> J.data)
    # return the result

    mat_J = J.reshape((x.size, -1))
    return mat_J

In [9]:
sol = pyequion.solve_solution({'NaHCO3': 10, 'CaCl2': 5})

In [10]:
J = cy_calc_cnv_jac_equilibrium_NaHCO3_CaCl2(25, sol.x)
J.shape

(15, 15)

['NaHCO3',
 'OH-',
 'H+',
 'Na+',
 'NaOH',
 'HCO3-',
 'CO3--',
 'NaCO3-',
 'Na2CO3',
 'CO2',
 'Ca++',
 'CaOH+',
 'CaCO3',
 'CaHCO3+',
 'Cl-',
 'H2O']

['NaHCO3',
 'OH-',
 'H+',
 'Na+',
 'NaOH',
 'HCO3-',
 'CO3--',
 'NaCO3-',
 'Na2CO3',
 'CO2',
 'Ca++',
 'CaOH+',
 'CaCO3',
 'CaHCO3+',
 'Cl-',
 'H2O']